In [ ]:
import monologue
#we use this to get fenced stuff out
from monologue.core.utils.ops import parse_fenced_code_blocks
#saves typing, fairly basic- generate pydantic types from example data
from monologue.core.utils.ops import pydantic_type_generator
#the entities we will create extend this which does the repr and pulls out some metadata
from monologue.entities import AbstractEntity


## Illustrate the type representation in logs

In [ ]:
from pydantic import Field
class MyEntity(AbstractEntity):
    code: str = Field(is_key=True)
    created_at: str

my_entity = MyEntity(code='test', created_at= "2023-01-01")
my_entity

## Show how the Columnar Store works

In [ ]:
from monologue.core.data.stores import ColumnarDataStore
from monologue.entities.examples import NycTripEvent
store = ColumnarDataStore(NycTripEvent)
store

### how we add data to the store

In [ ]:
# import pandas as pd
# data = pd.read_csv("/Users/sirsh/Downloads/nyc_trip_data_sample.csv") 
# store.add(data)

### load the tool and ask questions

In [ ]:
tool = store.as_tool()
tool

In [ ]:
tool.run("What is least popular destination in new york city? Who has travelled there?")

# Vector Store Loading

### bios

In [1]:
import pandas as pd
from monologue.core.data.stores import VectorDataStore
from monologue.core.utils.ops import pydantic_to_pyarrow_schema
from monologue.entities.examples import *

In [3]:
data = pd.read_csv("/Users/sirsh/Downloads/marvel_bios.csv").rename(columns={'entity_key':'id'})
data['id'] = data['id'].map(lambda x: x.replace('"',''))
data['name'] = data['id'].map(lambda x: x.split('_')[0])

data['doc_id'] = data['name']
data.tail()

,entity_type,id,text,name,doc_id
8104,people,Kaluu_12,Since then Doctor Strange has purged himself o...,Kaluu,Kaluu
8105,people,Kaluu_13,Kaluu was later contacted via crystal ball by ...,Kaluu,Kaluu
8106,people,Kaluu_14,"Kaluu is a powerful black magician, considered...",Kaluu,Kaluu
8107,people,Kaluu_15,Expert Occultist/Expert Magical Knowledge: Dur...,Kaluu,Kaluu
8108,people,Kaluu_16,Skilled Martial Artist: Kaluu is a skilled ath...,Kaluu,Kaluu


In [ ]:
store = VectorDataStore(AvengingPassengersInstruct)
data = [AvengingPassengersInstruct(**d) for d in data.to_dict('records')]
result = store.add(data)

2023-10-02 22:25:50.823 | WARNING  | monologue.core.data.vectors.lance:__init__:36 - Table does not exist - creating it from schema <class 'monologue.entities.examples.MyriadExamples.AvengingPassengersInstruct'>


load INSTRUCTOR_Transformer
'NoneType' object has no attribute 'cadam32bit_grad_fp32'


2023-10-02 22:25:56.381 | INFO     | monologue.core.data.stores.VectorStore:add:86 - Adding 8109 to examples_AvengingPassengersInstruct...


max_seq_length  512


In [ ]:
store.load()

In [ ]:
store("Who is Hank Pym?")

In [ ]:
store("What can you tell me about captain america? What was his real name?")

### places

In [ ]:
data = pd.read_csv("/Users/sirsh/Downloads/nyc_zones.csv").drop(columns='id',index=1).rename(columns={'entity_key':'id'})
data['doc_id'] = data['id']
data['name'] = data['id']
data.head()

In [ ]:
records = [Places(**d) for d in data.to_dict('records')]
#store.add(data)

In [ ]:
store = VectorDataStore(Places)
e = store.add(records[:10],plan=False)

In [ ]:
store.load()

In [ ]:
store("What can you tell me about civil airport in East Elmhurst Queens?")

In [ ]:
store.query_index("What can you tell me about civil airport in East Elmhurst Queens?")